In [1]:
# Import Libraries
import nltk
import pandas as pd
import pickle as pk
import ast

In [2]:
def opinion_sentiment(aspects):    
    aspects = ast.literal_eval(aspects)
    aspect_opinion_word = {}

    for aspect in aspects:
        aspect_opinion_word.setdefault(aspect[1], [0,0])
        word_senti = word_sentiment(aspect[0])
        if word_senti > 0: # Positive 
            aspect_opinion_word[aspect[1]][0] += word_senti
        elif word_senti < 0: # Negative
            aspect_opinion_word[aspect[1]][1] += word_senti

    # Structure : {aspect: [Positive, Negative]}
    return aspect_opinion_word

In [3]:
def word_sentiment(single_word):
    # Gets synonyms of a word from wordnet
    word_syn = nltk.corpus.wordnet.synsets(single_word)
    
    if len(word_syn)!=0:
        word = word_syn[0].name()
        # Gets sentiment word of a word in numerical value
        senti_score = nltk.corpus.sentiwordnet.senti_synset(word)
        
        if senti_score.pos_score() > senti_score.neg_score():
            return senti_score.pos_score()
        elif senti_score.pos_score() < senti_score.neg_score():
            return -(senti_score.neg_score())
        else:
            return 0
    return 0

In [4]:
# Import processed sample data from folder
infile = open("Stored Data/sample_reviews.pickle", "rb")
data = pk.load(infile)
infile.close()

data.head(10)

,Hotel,Reviews,Date,Score
0,Hotel Arena,Only the park outside of the hotel was beauti...,8/3/2017,2.9
1,Hotel Arena,No real complaints the hotel was great great ...,8/3/2017,7.5
2,K K Hotel George,Very comfortable beds smart bathroom good sho...,8/3/2017,9.6
3,Apex Temple Court Hotel,Everything,8/3/2017,10.0
4,Apex Temple Court Hotel,Great Hotel with wonderful staff and good cui...,8/3/2017,7.9
5,The Park Grand London Paddington,Location,8/3/2017,5.4
6,The Park Grand London Paddington,The front desk staff were very helpful and pl...,8/3/2017,7.5
7,The Park Grand London Paddington,Friendly atmosphere beds comfortable,8/3/2017,8.3
8,Park Plaza County Hall London,Just the location and view,8/3/2017,5.0
9,Park Plaza County Hall London,Around the corner from the London eye and use...,8/3/2017,7.5


In [8]:
# Pre-processed aspects gotten from Java Script (Enhanced Dependencies)
data['Aspect'] = pd.read_csv("Stored Data\java_output.txt", header=None, delimiter="\t", names = ["Aspects"])
data.head(10)

,Hotel,Reviews,Date,Score,Aspect
0,Hotel Arena,Only the park outside of the hotel was beauti...,8/3/2017,2.9,"[(""beautiful"",""park"")]"
1,Hotel Arena,No real complaints the hotel was great great ...,8/3/2017,7.5,"[(""real"",""complaints""),(""surroundings"",""compla..."
2,K K Hotel George,Very comfortable beds smart bathroom good sho...,8/3/2017,9.6,"[(""very comfortable"",""beds""),(""smart"",""bathroo..."
3,Apex Temple Court Hotel,Everything,8/3/2017,10.0,[]
4,Apex Temple Court Hotel,Great Hotel with wonderful staff and good cui...,8/3/2017,7.9,"[(""wonderful"",""staff""),(""good"",""cuisine"")]"
5,The Park Grand London Paddington,Location,8/3/2017,5.4,[]
6,The Park Grand London Paddington,The front desk staff were very helpful and pl...,8/3/2017,7.5,"[(""front"",""staff""),(""staff"",""desk""),(""very hel..."
7,The Park Grand London Paddington,Friendly atmosphere beds comfortable,8/3/2017,8.3,"[(""Friendly"",""beds""),(""beds"",""atmosphere"")]"
8,Park Plaza County Hall London,Just the location and view,8/3/2017,5.0,[]
9,Park Plaza County Hall London,Around the corner from the London eye and use...,8/3/2017,7.5,"[(""was"",""pickup""),(""cruise"",""river""),(""eating""..."


In [9]:
data['Sentiment'] = data['Aspect'].apply(lambda x: opinion_sentiment(x))
data.head()

,Hotel,Reviews,Date,Score,Aspect,Sentiment
0,Hotel Arena,Only the park outside of the hotel was beauti...,8/3/2017,2.9,"[(""beautiful"",""park"")]","{'park': [0.75, 0]}"
1,Hotel Arena,No real complaints the hotel was great great ...,8/3/2017,7.5,"[(""real"",""complaints""),(""surroundings"",""compla...","{'offer': [0, 0], 'site': [0, 0], 'quality': [..."
2,K K Hotel George,Very comfortable beds smart bathroom good sho...,8/3/2017,9.6,"[(""very comfortable"",""beds""),(""smart"",""bathroo...","{'bathroom': [0, -0.625], 'delight': [0, 0], '..."
3,Apex Temple Court Hotel,Everything,8/3/2017,10.0,[],{}
4,Apex Temple Court Hotel,Great Hotel with wonderful staff and good cui...,8/3/2017,7.9,"[(""wonderful"",""staff""),(""good"",""cuisine"")]","{'cuisine': [0.5, 0], 'staff': [0.75, 0]}"


In [10]:
# Store the aspect sample data
outfile = open("Stored Data/sample_review_w_aspects.pickle", "wb")
pk.dump(data, outfile)
outfile.close()

data.to_csv(r'Stored Data/sample_review_w_aspects.csv')